In [15]:
import os
import os.path as osp
import json
from argparse import ArgumentParser
from glob import glob

import torch
import cv2
from torch import cuda
from model import EAST
from tqdm import tqdm

import numpy
import sys
sys.path.append('../')
from inference import do_inference
from deteval import calc_deteval_metrics

In [16]:
def parse_args():
    parser = ArgumentParser()

    # Conventional args
    parser.add_argument('--data_dir', default='../../input/data/merged-dataset/')
    parser.add_argument('--model_dir', default='../trained_models/')
    parser.add_argument('--output_dir', default='./')

    parser.add_argument('--device', default='cuda' if cuda.is_available() else 'cpu')
    parser.add_argument('--input_size', type=int, default=1024)
    parser.add_argument('--batch_size', type=int, default=1)

    args = parser.parse_args('')

    if args.input_size % 32 != 0:
        raise ValueError('`input_size` must be a multiple of 32')

    return args
args = parse_args()
print(args)

Namespace(batch_size=1, data_dir='../../input/data/merged-dataset/', device='cuda', input_size=1024, model_dir='../trained_models/', output_dir='./')


In [17]:
# Initialize model
model = EAST(pretrained=False).to(args.device)
# Get paths to checkpoint files
ckpt_fpath = osp.join(args.model_dir, 'latest.pth')

if not osp.exists(args.output_dir):
    os.makedirs(args.output_dir)

print('Inference in progress')

ufo_result = dict(images=dict())
split_result = do_inference(model, ckpt_fpath, args.data_dir, args.input_size,
                                args.batch_size, split='images')
ufo_result['images'].update(split_result['images'])

output_fname = 'pred.json'
with open(osp.join(args.output_dir, output_fname), 'w') as f:
    json.dump(ufo_result, f, indent=4)

Inference in progress


100%|██████████| 2563/2563 [16:06<00:00,  2.65it/s]
